In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os
from datetime import datetime
import pytz
from pathlib import Path
import sklearn as sk
import tensorflow as tf
import torch
import torch.nn as nn
from torchvision.io import read_image
import time
import timeit
import random
import torchvision.models as models
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import sklearn.metrics
import csv
import timeit

%matplotlib inline

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls 'drive/MyDrive/DS5220: Final Project/logs/VGG_11'

20240425_192641_logs/


In [ ]:
BASE_LOGS_PATH = 'drive/MyDrive/DS5220: Final Project/logs/VGG_11'

In [ ]:
# https://pytorch.org/docs/stable/notes/mps.html

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
## Data cleaning & preprocessing

CIFAR100_ROOT_PATH='drive/MyDrive/DS5220: Final Project'

class Cifar100():
    def __init__(self,
                 calculate_mean_and_std = False):

        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

        self.BATCH_SIZE = 128
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)])

        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True,
                                transform=self.transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                train=False,
                                transform=self.transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                               num_workers=0,
                               batch_size=self.BATCH_SIZE,
                               shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                              num_workers=0,
                              batch_size=self.BATCH_SIZE,
                              shuffle=False)

    def calculate_mean_and_std(self):

        train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                download=True,
                                train=True)
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        _mean = np.mean(x, axis=(0, 1))/255
        _std = np.std(x, axis=(0, 1))/255
        _mean = _mean.tolist()
        _std = _std.tolist()

        return _mean, _std



# data augmentation
class Cifar100WithAugmentation():
    def __init__(self, calculate_mean_and_std=False):
        CIFAR100_ROOT_PATH = 'drive/MyDrive/DS5220: Final Project'
        if calculate_mean_and_std:
            self.mean, self.std = self.calculate_mean_and_std()
        else:
            self.mean, self.std = (0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)

        self.BATCH_SIZE = 128

        self.train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.eval_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])
        self.train_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                               download=True,
                                               train=True,
                                               transform=self.train_transform)

        self.eval_dataset = CIFAR100(root=CIFAR100_ROOT_PATH,
                                              train=False,
                                              transform=self.eval_transform)

        self.train_data_loader = DataLoader(dataset=self.train_dataset,
                                            num_workers=4,
                                            batch_size=self.BATCH_SIZE,
                                            shuffle=True)

        self.eval_data_loader = DataLoader(dataset=self.eval_dataset,
                                           num_workers=4,
                                           batch_size=self.BATCH_SIZE,
                                           shuffle=False)

    def calculate_mean_and_std(self):
        train_dataset = CIFAR100(root='drive/MyDrive/DS5220: Final Project',
                                          download=True,
                                          train=True)
        x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
        mean = np.mean(x, axis=(0, 1)) / 255
        std = np.std(x, axis=(0, 1)) / 255
        return mean.tolist(), std.tolist()


cf100 = Cifar100(calculate_mean_and_std=True)
cf100_w_aug = Cifar100WithAugmentation(calculate_mean_and_std=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# model config for VGG11
config = {
    'dataloader' : cf100_w_aug,
    'model': models.vgg11(),
    'epochs': 40,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device' : device,
    'run_time' : None
}

model = config['model']
model.to(device)
EPOCHS = config['epochs']
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")
else:
    print(f"Directory '{config['log_dir']}' already exists")

metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    training_start_time = timeit.default_timer()
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []
        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                #  top-1 accuracy
                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                #  top-5 accuracy
                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)

                eval_accuracies.append(top1_accuracy)

        end_time = timeit.default_timer()
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")
    training_end_time = timeit.default_timer()
    training_duration = training_end_time - training_start_time
    config['run_time'] = training_duration
config_path = os.path.join(config['log_dir'], 'config.txt')
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')


torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))


Directory 'drive/MyDrive/DS5220: Final Project/logs/VGG_11/20240425_213841_logs' created
Epoch 1/40
Train Loss: 4.6586, Train Accuracy: 2.36%
Eval Loss: 4.1865, Eval Accuracy: 5.06%, Top-1 Accuracy: 5.06%, Top-5 Accuracy: 20.02%
Epoch 2/40
Train Loss: 4.0202, Train Accuracy: 6.82%
Eval Loss: 3.7570, Eval Accuracy: 10.96%, Top-1 Accuracy: 10.96%, Top-5 Accuracy: 32.18%
Epoch 3/40
Train Loss: 3.7036, Train Accuracy: 11.25%
Eval Loss: 3.4470, Eval Accuracy: 15.78%, Top-1 Accuracy: 15.78%, Top-5 Accuracy: 43.41%
Epoch 4/40
Train Loss: 3.4066, Train Accuracy: 16.49%
Eval Loss: 3.2012, Eval Accuracy: 19.92%, Top-1 Accuracy: 19.92%, Top-5 Accuracy: 50.00%
Epoch 5/40
Train Loss: 3.1489, Train Accuracy: 21.10%
Eval Loss: 2.9365, Eval Accuracy: 24.99%, Top-1 Accuracy: 24.99%, Top-5 Accuracy: 57.03%
Epoch 6/40
Train Loss: 2.9292, Train Accuracy: 25.42%
Eval Loss: 2.6702, Eval Accuracy: 31.36%, Top-1 Accuracy: 31.36%, Top-5 Accuracy: 63.88%
Epoch 7/40
Train Loss: 2.7432, Train Accuracy: 28.91%
Eva

In [ ]:
# model for custom vgg with addtional drop
class CustomVGG11(nn.Module):
    def __init__(self, num_classes=100, dropout_rate=0.5):
        super(CustomVGG11, self).__init__()
        self.vgg11 = models.vgg11(pretrained=False)
        self.vgg11.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        return self.vgg11(x)

config = {
    'dataloader': cf100_w_aug,
    'model': CustomVGG11(num_classes=100, dropout_rate=0.5),
    'epochs': 70,
    'optimizer': 'SGD',
    'learning_rate': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'loss': nn.CrossEntropyLoss(),
    'log_dir': os.path.join(BASE_LOGS_PATH, time.strftime('%Y%m%d_%H%M%S') + '_logs'),
    'device': device,
    'run_time': None
}

model = config['model']
model.to(config['device'])
criterion = config['loss']
optimizer = torch.optim.SGD(model.parameters(),
                            lr=config['learning_rate'],
                            momentum=config['momentum'],
                            weight_decay=config['weight_decay'])

if not os.path.exists(config['log_dir']):
    os.makedirs(config['log_dir'])
    print(f"Directory '{config['log_dir']}' created")

metrics_path = os.path.join(config['log_dir'], 'training_metrics.csv')
with open(metrics_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy', 'Eval Loss', 'Eval Accuracy', 'Top-1 Accuracy', 'Top-5 Accuracy', 'Epoch Time (seconds)'])
    for epoch in range(config['epochs']):
        start_time = timeit.default_timer()
        train_losses = []
        train_accuracies = []

        model.train()
        for images, labels in config['dataloader'].train_data_loader:
            images, labels = images.to(config['device']), labels.to(config['device'])
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).float().mean().item()
            train_accuracies.append(accuracy)

        eval_losses = []
        eval_accuracies = []
        top1_accuracies = []
        top5_accuracies = []
        model.eval()
        with torch.no_grad():
            for images, labels in config['dataloader'].eval_data_loader:
                images, labels = images.to(config['device']), labels.to(config['device'])
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_losses.append(loss.item())

                _, predicted = torch.max(outputs, 1)
                top1_accuracy = (predicted == labels).float().mean().item()
                top1_accuracies.append(top1_accuracy)

                _, top5_preds = outputs.topk(5, dim=1)
                top5_correct = top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))
                top5_accuracy = top5_correct.float().sum(1).ge(1).float().mean().item()
                top5_accuracies.append(top5_accuracy)
                eval_accuracies.append(top1_accuracy)

        end_time = timeit.default_timer()
        epoch_duration = end_time - start_time
        writer.writerow([
            epoch + 1,
            np.mean(train_losses),
            np.mean(train_accuracies) * 100,
            np.mean(eval_losses),
            np.mean(eval_accuracies) * 100,
            np.mean(top1_accuracies) * 100,
            np.mean(top5_accuracies) * 100,
            epoch_duration
        ])
        print(f"Epoch {epoch+1}/{config['epochs']}")
        print(f"Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_accuracies) * 100:.2f}%")
        print(f"Eval Loss: {np.mean(eval_losses):.4f}, Eval Accuracy: {np.mean(eval_accuracies) * 100:.2f}%, Top-1 Accuracy: {np.mean(top1_accuracies) * 100:.2f}%, Top-5 Accuracy: {np.mean(top5_accuracies) * 100:.2f}%")


torch.save(model.state_dict(), os.path.join(config['log_dir'], 'model_state_dict.pth'))
torch.save(model, os.path.join(config['log_dir'], 'complete_model.pth'))

config_path = os.path.join(config['log_dir'], 'config.txt')
with open(config_path, 'w') as f:
    for key, value in config.items():
        f.write(f'{key}: {value}\n')



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Directory 'drive/MyDrive/DS5220: Final Project/logs/VGG_11/20240425_232158_logs' created
Epoch 1/40
Train Loss: 4.5352, Train Accuracy: 1.45%
Eval Loss: 4.4461, Eval Accuracy: 2.26%, Top-1 Accuracy: 2.26%, Top-5 Accuracy: 10.21%
Epoch 2/40
Train Loss: 4.4201, Train Accuracy: 2.36%
Eval Loss: 4.3186, Eval Accuracy: 4.09%, Top-1 Accuracy: 4.09%, Top-5 Accuracy: 14.97%
Epoch 3/40
Train Loss: 4.1597, Train Accuracy: 4.39%
Eval Loss: 3.9864, Eval Accuracy: 6.71%, Top-1 Accuracy: 6.71%, Top-5 Accuracy: 24.67%
Epoch 4/40
Train Loss: 3.9089, Train Accuracy: 7.45%
Eval Loss: 3.7630, Eval Accuracy: 10.49%, Top-1 Accuracy: 10.49%, Top-5 Accuracy: 33.73%
Epoch 5/40
Train Loss: 3.6577, Train Accuracy: 11.55%
Eval Loss: 3.4658, Eval Accuracy: 15.09%, Top-1 Accuracy: 15.09%, Top-5 Accuracy: 40.89%
Epoch 6/40
Train Loss: 3.4197, Train Accuracy: 15.29%
Eval Loss: 3.2118, Eval Accuracy: 19.59%, Top-1 Accuracy: 19.59%, Top-5 Accuracy: 49.13%
Epoch 7/40
Train Loss: 3.2207, Train Accuracy: 19.00%
Eval Loss